In [ ]:
import torch

# 저장된 모델 불러오기
model_path = "combined_generator_256.pt"  # 저장된 모델 파일 경로
model = torch.load(model_path, map_location='cpu')  # CPU로 불러올 경우
model.eval()  # 추론 모드로 설정


C:\Users\USER\AppData\Local\Temp\ipykernel_17992\2427109006.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(final_model_path, map_location='cpu')  # C

RecursiveScriptModule(
  original_name=Generator
  (condition_embedding): RecursiveScriptModule(
    original_name=ConditionEmbedding
    (linear): RecursiveScriptModule(original_name=Linear)
  )
  (ae_encoder): RecursiveScriptModule(
    original_name=Encoder
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
    (conv2): RecursiveScriptModule(original_name=Conv2d)
    (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
    (conv3): RecursiveScriptModule(original_name=Conv2d)
    (bn3): RecursiveScriptModule(original_name=BatchNorm2d)
    (conv4): RecursiveScriptModule(original_name=Conv2d)
    (bn4): RecursiveScriptModule(original_name=BatchNorm2d)
    (conv5): RecursiveScriptModule(original_name=Conv2d)
    (bn5): RecursiveScriptModule(original_name=BatchNorm2d)
    (conv6): RecursiveScriptModule(original_name=Conv2d)
    (bn6): RecursiveScriptModule(original_name=BatchNorm2d)
    (conv7): RecursiveScriptModule(or

In [14]:
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader

image_path = "TEST"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=image_path, transform=transform)
train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

for step, batch in enumerate(train_dataloader):
    input_image, labels = batch
    c = F.one_hot(labels, num_classes=8).float()
    break


In [13]:
import os
import random

def assign_labels(root_dir):
    """
    폴더명 -> 라벨 인덱스 맵핑
    """
    subdirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    subdirs.sort()
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(subdirs)}
    return class_to_idx

def split_dataset(root_dir, train_ratio=0.8):
    class_to_idx = assign_labels(root_dir)

    train_data = []
    test_data = []

    for cls_name, cls_idx in class_to_idx.items():
        subdir_path = os.path.join(root_dir, cls_name)
        if os.path.isdir(subdir_path):
            images = [os.path.join(subdir_path, file) for file in os.listdir(subdir_path)
                      if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
            random.shuffle(images)
            split_idx = int(len(images) * train_ratio)

            train_data.extend([(img_path, cls_idx) for img_path in images[:split_idx]])
            test_data.extend([(img_path, cls_idx) for img_path in images[split_idx:]])
    return train_data, test_data

class CustomDataset(datasets.VisionDataset):
    def __init__(self, data_list, transform=None):
        super(CustomDataset, self).__init__(root=None, transform=transform)
        self.data_list = data_list
        self.transform = transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        img_path, label = self.data_list[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

def get_dataloader(root_dir, batch_size, image_size, train_ratio=0.8):
    train_data, test_data = split_dataset(root_dir, train_ratio)
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

    train_dataset = CustomDataset(train_data, transform=transform)
    test_dataset = CustomDataset(test_data, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    return train_loader, test_loader

In [ ]:
get_dataloader

In [15]:
dataset_path = 'TEST'
batch_size = 1
image_size = 256

with torch.no_grad():
    # train_loader, valid_loader = get_dataloader(dataset_path, batch_size, image_size)
    reconstructed_image = model(input_image, c)

In [16]:
output_image = reconstructed_image.squeeze(0)
output_image = output_image.permute(1, 2, 0).cpu().numpy()
output_image = (output_image * 255).astype("uint8")

AttributeError: 'tuple' object has no attribute 'squeeze'

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

image = Image.open(image_path).convert("RGB")

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Reconstructed Image")
plt.imshow(output_image)
plt.axis("off")

plt.show()